# 6 run time 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [41]:
import time
starttime = time.time()

import itertools
import math
import numpy as np
import numpy.fft as fft
import pandas as pd

from joblib import Parallel, delayed
from numba import njit
from scipy.stats import zscore
from tqdm import tqdm
from saxpy.znorm import znorm
from ast import literal_eval
from os.path import exists

from saxpy.znorm import znorm
from saxpy.paa import paa
from saxpy.alphabet import cuts_for_asize
from saxpy.sax import ts_to_string

slack = 0.6

file = 'C:\\Users\\jbzp18481526321\\datasets\\original\\npo141.csv'
ts_csv= pd.read_csv(file, index_col=0)
ts_origin = ts_csv.values   #csv to numpy
ts_origin = ts_origin[0:200000]

paa_length = 4
paa_size = len(ts_origin)//paa_length
ts = ts_origin[0:(paa_size * paa_length)]            #截取需要实验的TS
ts = list(itertools.chain.from_iterable(ts))
paa_win_size = paa_length*40

def apply_sax(ts, paa_win_size, paa_length, alphabet_size):           #paa_length是原时间序列长度和新时间序列长度之比，即n/w       
    sax_sequence = []
    cuts = cuts_for_asize(alphabet_size)
    ts_znorm = znorm(ts, 0.01)              #z_threshold 取 0.01         #先将整个时间序列离散化
    for t in np.arange(0, len(ts) - paa_win_size + 1, paa_win_size):
        ts_win = znorm(ts[t:(t+paa_win_size)])          #取窗口
        paa_rep = paa(ts_win, 100)            #取平均值   
        sax_word = list(ts_to_string(paa_rep, cuts))   #离散化
        sax_sequence.append(sax_word)
        sax_sequence = list(itertools.chain.from_iterable(sax_sequence))
    return sax_sequence


sax_sequence = apply_sax(ts,paa_size, 4, 13)


def extract_modified_bs_sequence(sax_sequence):
    
    bs_sequence = []
    bs_lengths = []
    curr_len = paa_length
    for i in range(len(sax_sequence)):
        curr_bs = sax_sequence[i]
        if i < len(sax_sequence)-1:
            next_bs = sax_sequence[i+1]
        else:
            next_bs = ''
        if curr_bs == next_bs:
            curr_len = curr_len + paa_length      #
        else:
            bs_sequence.append(curr_bs)  #直到出现第一次不相等的情形，将相邻的sax symbol合并
            bs_lengths.append(curr_len)
            curr_len = paa_length
    return bs_sequence, bs_lengths


#上述得到的bs_lengths并不是指针，为此我们用下面的函数
def generate_bs_pointers(bs_lengths):
    bs_pointers = []
    start_pointer = 0
    for bs_len_item in bs_lengths:
        end_pointer = start_pointer + bs_len_item       #bs_len_item是一个bs连续重复出现的次数 
        pointer_list = list(range(start_pointer, end_pointer))       #方便后续索引，motif的长度变成了window的长度加上bs_len_item的
        bs_pointers.append(pointer_list)                             #得到列表的列表
        start_pointer = start_pointer + bs_len_item
    return bs_pointers 

bs_sequence, bs_lengths = extract_modified_bs_sequence(sax_sequence)
bs_pointer = generate_bs_pointers(bs_lengths)  

win_size = 7
pattern_list = []
patterns = []
for i in range(len(bs_sequence)-win_size+1):
    saxpattern=[]
    sax_win = bs_sequence[i:i+win_size]
    for j in range(len(bs_sequence)-win_size+1):
        pattern = []
        cur_word = bs_sequence[j:j+win_size]
        if sax_win == cur_word:
            cur_word_location = list(itertools.chain.from_iterable(bs_pointer[j:j+win_size]))                                    
            saxpattern.append(cur_word_location)
            pattern.append(cur_word)
    if len(saxpattern) > 1:
        pattern_list.append(saxpattern)

motif_head = pattern_list[0][0][0]
motif_length = len(pattern_list[0][0])


ts_center = znorm(ts[motif_head:motif_head + motif_length])
r = 4
motifs = []
for i in range(len(ts)-motif_length+1):
    ts_win = znorm(ts[i:i+motif_length])
    dist = math.dist(ts_center, ts_win)    
    if dist < r:
        motifs.append(i)
        
def filter_non_trivial_matches(motif_set, m):
    #对窗口长度为m的motif set，排除掉trivial—match的subsequence（删掉位置靠后的一个）
    # filter trivial matches
    non_trivial_matches = []
    last_offset = - m
    for offset in np.sort(motif_set):           #将motif set的index进行排序后，判断相邻两数之间距是否大于m/2
        if offset > last_offset + m / 2:
            non_trivial_matches.append(offset)   
            last_offset = offset

    return np.array(non_trivial_matches)

motifs_nonoverlapping = filter_non_trivial_matches(motifs,motif_length )

endtime = time.time()
runtime = endtime-starttime

print(runtime)

31.22040295600891
